# 1. 회귀문제

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor

from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, recall_score, f1_score, precision_score
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from boruta import BorutaPy

## 데이터 전처리

#### 1. 데이터 불러오기 및 Unnamed: 0 변수 제거

In [2]:
bicycle = pd.read_csv("./regression.csv")

In [3]:
bicycle.drop(labels="Unnamed: 0", axis=1, inplace=True)

#### 2. count를 y값, 나머지를 X값으로 처리

In [4]:
x = bicycle.drop(["count"], axis=1)
y = bicycle["count"]

#### 3. 학습, 테스트 데이터 8:2비율로 분할 
* random state=1 으로 지정

In [5]:
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.8, random_state=1)

#### 4. X 데이터 Standard Scale 진행

In [6]:
scaler = StandardScaler()
scaler.fit(train_x)

train_x = pd.DataFrame(scaler.transform(train_x), columns=train_x.columns)
test_x = pd.DataFrame(scaler.transform(test_x), columns=test_x.columns)

#### 5. 학습데이터 X 통계량 출력(describe 함수 사용)

In [7]:
train_x.describe()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
count,1.073000e+03,1.073000e+03,1.073000e+03,1.073000e+03,1.073000e+03,1.073000e+03,1.073000e+03,1.073000e+03,1.073000e+03
mean,9.301869e-17,5.041013e-16,-3.212714e-17,-5.690798e-17,1.024344e-16,-4.449170e-17,-3.794728e-16,7.573935e-17,-2.506538e-17
std,1.000466e+00,1.000466e+00,1.000466e+00,1.000466e+00,1.000466e+00,1.000466e+00,1.000466e+00,1.000466e+00,1.000466e+00
min,-1.754004e+00,-2.664005e+00,-1.781313e-01,-1.819309e+00,-2.226817e+00,-2.321415e+00,-1.861060e+00,-1.518514e+00,-1.517693e+00
25%,-8.684045e-01,-7.304412e-01,-1.781313e-01,-7.958030e-01,-7.741268e-01,-8.780980e-01,-6.887126e-01,-7.015717e-01,-7.106802e-01
50%,1.719477e-02,-2.210593e-02,-1.781313e-01,-6.472713e-02,-7.282824e-02,2.835114e-01,-2.608163e-02,-1.988378e-01,-3.071738e-01
75%,9.027940e-01,6.287968e-01,-1.781313e-01,6.663488e-01,7.787486e-01,1.010387e+00,6.365494e-01,3.981586e-01,4.325879e-01
max,1.640793e+00,2.485784e+00,5.613836e+00,4.029298e+00,2.381717e+00,1.020820e+00,4.357477e+00,6.116756e+00,3.996894e+00


## 모델 학습 및 평가

#### 1. RandomForest, XGBoost, LightGBM 3가지 모델에 대해 학습데이터를 사용하여 최적의 하이퍼파라미터 탐색
- GridSearchCV cv=5를 통해 탐색
- Random Forest 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , random_state=1 
- XGBoost 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , gamma = 0.5,1,2  , random_state=1 
- LightGBM 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , random_state=1 
- 평가지표 : R-Square

In [8]:
rfr = RandomForestRegressor()
xgboost = XGBRegressor()
lightgbm = LGBMRegressor()

param_rfr = {
    'max_depth': [3,5,10],
    'n_estimators': [100,200,300],
    'random_state':[1]
}
param_xgb = {
    'max_depth' : [3,5,10],
    'n_estimators' : [100,200,300],
    'learning_rate' : [0.001, 0.01, 0.1, 1],
    'gamma' : [0.5, 1, 2],
    'random_state' :[1]
}
param_lgbm = {
    'max_depth' : [3,5,10],
    'n_estimators' : [100,200,300],
    'learning_rate' : [0.001,0.01,0.1,1],
    'random_state' : [1]
}

gscv_rfr = GridSearchCV(estimator=rfr, param_grid = param_rfr, scoring='r2', cv=5)
gscv_xgb = GridSearchCV(estimator=xgboost, param_grid = param_xgb, scoring='r2', cv=5)
gscv_lgbm = GridSearchCV(estimator=lightgbm, param_grid = param_lgbm, scoring='r2', cv=5)

gscv_rfr.fit(train_x, train_y)
gscv_xgb.fit(train_x, train_y)
gscv_lgbm.fit(train_x, train_y)

print(f"[RandomForest] R2 score = {gscv_rfr.best_score_} / parameters = {gscv_rfr.best_params_}")
print(f"[XGBoost] R2 score = {gscv_xgb.best_score_} / parameters = {gscv_xgb.best_params_}")
print(f"[lightGBM] R2 score = {gscv_lgbm.best_score_} / parameters = {gscv_lgbm.best_params_}")

[RandomForest] R2 score = 0.7693585584046533 / parameters = {'max_depth': 10, 'n_estimators': 100, 'random_state': 1}
[XGBoost] R2 score = 0.7820826406316188 / parameters = {'gamma': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'random_state': 1}
[lightGBM] R2 score = 0.78391999861395 / parameters = {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'random_state': 1}


#### 2. 학습데이터로 평가했을 때 가장 좋은 성능을 보인 하이퍼 파라미터값의RandomForest, XGBoost, LightGBM모델을 테스트 데이터로 평가
* 평가지표 R-Square, MSE
* 반올림 하여 소수점 셋째짜리 까지 계산

In [9]:
rfr_best = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=1)
xgboost_best = XGBRegressor(gamma=0.5, learning_rate=0.1, max_depth=5, n_estimators=300, random_state=1)
lightgbm_best = LGBMRegressor(learning_rate=0.1, max_depth=10, n_estimators=100, random_state=1)

rfr_best.fit(train_x, train_y)
xgboost_best.fit(train_x, train_y)
lightgbm_best.fit(train_x, train_y)

LGBMRegressor(max_depth=10, random_state=1)

In [10]:
print("[RandomForest] R-square : {:.3f}, MSE : {:.3f}".format(r2_score(test_y, rfr_best.predict(test_x)), mean_squared_error(test_y, rfr_best.predict(test_x))))
print("[XGBoost] R-square : {:.3f}, MSE : {:.3f}".format(r2_score(test_y, xgboost_best.predict(test_x)), mean_squared_error(test_y, xgboost_best.predict(test_x))))
print("[LightGBM] R-square : {:.3f}, MSE : {:.3f}".format(r2_score(test_y, lightgbm_best.predict(test_x)), mean_squared_error(test_y, lightgbm_best.predict(test_x))))

[RandomForest] R-square : 0.762, MSE : 1526.263
[XGBoost] R-square : 0.746, MSE : 1629.751
[LightGBM] R-square : 0.759, MSE : 1544.256


## 데이터 전처리

#### 1. 데이터 불러오기

In [11]:
wine = pd.read_csv("./classification.csv")

#### 2. type값의 white, red를 숫자로 변경. 이때 white은 0 red는 1로 변경


In [12]:
wine['type']=np.where(wine['type']=='white', 0, wine['type'])
wine['type']=np.where(wine['type']=='red', 1, wine['type'])

#### 3 type을 y, 나머지 변수를 X값으로 분할


In [13]:
wx = wine.drop(['type'], axis=1, inplace=False)
wy = wine['type'].astype('int')

#### 3 X, y 를 8: 2로 데이터 분할
- stratify를 통해 y값 클래스 비율 일정하게 분할
- random_state = 1로 지정

In [14]:
train_wx, test_wx, train_wy, test_wy = train_test_split(wx, wy, train_size=0.8, random_state=1, stratify=wy)

#### 4. X 데이터 Standard Scale 진행


In [15]:
scaler = StandardScaler()
scaler.fit(train_wx)

train_wx = pd.DataFrame(scaler.transform(train_wx), columns=train_wx.columns)
test_wx = pd.DataFrame(scaler.transform(test_wx), columns=test_wx.columns)

#### 5. 학습데이터 X 통계량 출력(describe 함수 사용)

In [16]:
train_wx.describe()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03,4.397000e+03
mean,-2.565355e-16,2.053874e-16,4.565120e-17,-1.730983e-16,1.457404e-16,9.065853e-17,-5.650851e-17,-2.888549e-17,1.308361e-14,8.957028e-16,-4.131080e-16,-8.604039e-16
std,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00,1.000114e+00
min,-3.222467e+00,-2.598173e+00,-1.590381e+00,-2.241634e+00,-1.026032e+00,-1.378994e+00,-1.655318e+00,-1.949450e+00,-2.546053e+00,-2.980231e+00,-2.084266e+00,-1.756277e+00
25%,-9.372983e-01,-6.390549e-01,-6.574173e-01,-4.871902e-01,-7.686988e-01,-5.270247e-01,-7.530840e-01,-6.657428e-01,-8.024685e-01,-6.798087e-01,-6.763825e-01,-8.397119e-01
50%,2.052857e-01,-1.688665e-01,-2.842318e-01,-6.612380e-02,-5.328097e-01,-2.626204e-01,-7.640875e-02,4.742779e-02,5.070277e-02,-5.807297e-02,-1.400458e-01,-1.731191e-01
75%,2.052857e-01,3.796867e-01,3.999417e-01,4.952981e-01,5.823022e-01,2.368100e-01,6.002665e-01,7.071106e-01,7.684500e-01,6.258363e-01,4.633329e-01,7.017840e-01
max,3.633038e+00,6.805595e+00,7.739257e+00,6.390228e+00,5.621750e+00,1.198811e+01,1.458489e+01,5.788451e+00,5.298383e+00,4.915813e+00,9.715140e+00,3.659790e+00


#### 6. PCA를 통한 변수 축약
- explained_variance_ratio_를 통해 90%가 넘어지는 시점을 기준으로 축약할 개수 선택
- 학습데이터를 이용해 진행

In [17]:
PCA_list=[]
for i in range(1,10):
    pca=sum(PCA(n_components=i).fit(train_wx).explained_variance_ratio_)
    print(i,'개의 잠재변수가 설명하는 분산은 전체 데이터의 {:.2f}%'.format(pca*100))

1 개의 잠재변수가 설명하는 분산은 전체 데이터의 25.35%
2 개의 잠재변수가 설명하는 분산은 전체 데이터의 47.31%
3 개의 잠재변수가 설명하는 분산은 전체 데이터의 61.00%
4 개의 잠재변수가 설명하는 분산은 전체 데이터의 69.87%
5 개의 잠재변수가 설명하는 분산은 전체 데이터의 76.75%
6 개의 잠재변수가 설명하는 분산은 전체 데이터의 82.41%
7 개의 잠재변수가 설명하는 분산은 전체 데이터의 87.10%
8 개의 잠재변수가 설명하는 분산은 전체 데이터의 91.54%
9 개의 잠재변수가 설명하는 분산은 전체 데이터의 95.32%


In [20]:
pca8 = PCA(n_components=8)

pca_train_wx=pca8.fit_transform(train_wx)
pca_test_wx = pca8.transform(test_wx)

#### 7. PCA 진행한 후의 학습, 테스트 데이터 X 평균 각각 계산
* 반올림 하여 소수점 셋째짜리 까지 계산

In [21]:
print("mean of train data : {:.3f}".format(np.mean(pca_train_wx)))
print("mean of test data : {:.3f}".format(np.mean(pca_test_wx)))

mean of train data : -0.000
mean of test data : -0.004


## 모델 학습 및 평가

#### 1. SVM, Decision Tree 2가지 모델에 대해 학습데이터를 사용하여 최적의 하이퍼파라미터 탐색
- GridSearchCV cv=5를 통해 탐색
- SVM 하이퍼파라미터 후보 : kernel=rbf, linear, poliy , C= 0.5, 1.5, 10, random_state=1
- Decision Tree 하이퍼파라미터 후보 : criterion : gini, entropy , max_depth=5,10,15,20,None , random_state=1
- 평가지표 : F1-Score

In [22]:
svm = SVC()
dt = tree.DecisionTreeClassifier()

param_svm = {
    'kernel' : ['rbf', 'linear', 'poly'],
    'C' : [0.5, 1.5, 10],
    'random_state': [1]
}
param_dt = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [5, 10, 15, 20, None],
    'random_state' : [1]
}

gscv_svm = GridSearchCV(svm, param_svm, scoring='f1', cv=5)
gscv_dt = GridSearchCV(estimator=dt, param_grid=param_dt, scoring='f1', cv=5)

gscv_svm.fit(pca_train_wx, train_wy)
gscv_dt.fit(train_wx, train_wy)

print(f"[SVM] F1 score = {gscv_svm.best_score_} / parameters = {gscv_svm.best_params_}")
print(f"[DT] F1 score = {gscv_dt.best_score_} / parameters = {gscv_dt.best_params_}")

[SVM] F1 score = 0.9892247157830854 / parameters = {'C': 1.5, 'kernel': 'rbf', 'random_state': 1}
[DT] F1 score = 0.9696867280306971 / parameters = {'criterion': 'entropy', 'max_depth': 10, 'random_state': 1}


#### 2. 학습데이터로 평가했을 때 가장 좋은 성능을 보인 하이퍼 파라미터값의 SVM Decision Tree 모델을 테스트 데이터로 평가
* 평가지표 Recall, Precision ,F1-Score
* 반올림 하여 소수점 셋째짜리 까지 계산

In [23]:
svm_best = SVC(C=1.5, kernel='rbf', random_state=1)
dt_best = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=1)

svm_best.fit(pca_train_wx, train_wy)
dt_best.fit(pca_train_wx, train_wy)

DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=1)

In [24]:
print("[SVM] Recall : {:.3f}, Precision : {:.3f}, F1-Score : {:.3f}".format(recall_score(test_wy, svm_best.predict(pca_test_wx)), precision_score(test_wy, svm_best.predict(pca_test_wx)), f1_score(test_wy, svm_best.predict(pca_test_wx))))
print("[DT] Recall : {:.3f}, Precision : {:.3f}, F1-Score : {:.3f}".format(recall_score(test_wy, dt_best.predict(pca_test_wx)), precision_score(test_wy, dt_best.predict(pca_test_wx)), f1_score(test_wy, dt_best.predict(pca_test_wx))))


[SVM] Recall : 0.978, Precision : 0.992, F1-Score : 0.985
[DT] Recall : 0.974, Precision : 0.960, F1-Score : 0.967
